## predict

In [4]:
# UltralyticsのYOLOv8を使用して画像推論を実行
from ultralytics import YOLO
import os 

model = YOLO(r"D:\prog_py\Yolo_trainer\runs\train\yolo12n-seg-pod\weights\best.pt")

In [ ]:
# 指定されたパラメータで画像推論を実行
print("推論を開始します...")

source_path = r"D:\prog_py\yolo12_detect\detect_images"
if os.path.exists(source_path):
    print("🔍 画像サイズ（32の倍数, 1500まで）で推論を実行しています...")

    # 32の倍数で512から1248まで
    grid_imgsz = list(range(512, 1248, 32))

    for imgsz in grid_imgsz:
        print(f"\n--- 画像サイズ: {imgsz} で推論中 ---")
        # 実験名を画像サイズごとに分けて保存することで各サイズごとに結果が見やすく、すべての画像サイズで推論できるようにする
        results = model.predict(
            source=source_path,           # 画像フォルダ: detect_images
            imgsz=imgsz,                  # 画像サイズを動的に変更
            save_txt=True,                # テキストファイル保存: True
            line_width=1,                 # バウンディングボックスの線の太さ: 1
            save=True,                    # 推論結果の画像を保存
            conf=0.25,                    # 信頼度のしきい値
            device='0',                   # デバイス（GPUが利用可能な場合は'0'に変更）
            project='runs/detect',        # 結果保存ディレクトリ
            name=f'yolo_detection_{imgsz}',# 画像サイズごとにディレクトリ分け
            retina_masks=True,
            agnostic_nms=True,
        )

print("✓ 推論が完了しました！")
print(f"結果は runs/detect/yolo_detection_＜imgsz＞ ディレクトリに保存されました")


推論を開始します...
🔍 画像サイズ（32の倍数, 1500まで）で推論を実行しています...

--- 画像サイズ: 512 で推論中 ---

image 1/82 D:\prog_py\yolo12_detect\detect_images\024_01_b.jpg: 512x288 55 pods, 105.1ms
image 2/82 D:\prog_py\yolo12_detect\detect_images\024_02_b.jpg: 512x224 59 pods, 63.3ms
image 3/82 D:\prog_py\yolo12_detect\detect_images\024_02_f.jpg: 512x288 32 pods, 17.7ms
image 4/82 D:\prog_py\yolo12_detect\detect_images\024_03_b.jpg: 512x320 46 pods, 72.8ms
image 5/82 D:\prog_py\yolo12_detect\detect_images\024_03_f.jpg: 512x256 22 pods, 68.0ms
image 6/82 D:\prog_py\yolo12_detect\detect_images\024_05_b.jpg: 512x256 38 pods, 16.1ms
image 7/82 D:\prog_py\yolo12_detect\detect_images\024_05_f.jpg: 512x256 35 pods, 14.7ms
image 8/82 D:\prog_py\yolo12_detect\detect_images\043_01_b.jpg: 512x352 31 pods, 122.7ms
image 9/82 D:\prog_py\yolo12_detect\detect_images\043_01_f.jpg: 512x416 29 pods, 93.0ms
image 10/82 D:\prog_py\yolo12_detect\detect_images\043_05_b.jpg: 512x288 20 pods, 18.5ms
image 11/82 D:\prog_py\yolo12_detect\detec

In [3]:

model = YOLO(r"D:\prog_py\yolo12_detect\runs\train\train10\weights\best.pt")
# dataset.yamlのパス
dataset_yaml_path = r"D:\Data\pods\2025_yolo_pod\labels\YOLODataset\dataset.yaml"

# 画像サイズ（imgsz）はmax stride 32の倍数でグリッドサーチ
if os.path.exists(dataset_yaml_path):
    print("🔍 画像サイズ（32の倍数, 1500まで）で検証を実行しています...")

    # 32の倍数で500から1504まで
    grid_imgsz = list(range(512, 1248, 32))

    for imgsz in grid_imgsz:
        print(f"\n--- 画像サイズ: {imgsz} で検証中 ---")
        validation_results = model.val(
            data=dataset_yaml_path,
            imgsz=imgsz,
            plots=True,
            batch=8,
            save_json=True,
            device='0',
            project='runs/val',
            name=f'val_imgs_{imgsz}',
            verbose=True,
        )


else:
    print("❌ データセット設定ファイルが見つからないため、検証をスキップします。")


🔍 画像サイズ（32の倍数, 1500まで）で検証を実行しています...

--- 画像サイズ: 512 で検証中 ---
Ultralytics 8.3.223  Python-3.12.6 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11l-seg summary (fused): 203 layers, 27,585,363 parameters, 0 gradients, 131.8 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 899.1118.3 MB/s, size: 190.6 KB)
val: Scanning \\?\D:\Data\pods\2025_yolo_pod\labels\YOLODataset\labels\val.cache... 13 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 13/13 12.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.1it/s 14.8s4.1s
                   all         13        554      0.667      0.578      0.586      0.313       0.65      0.563       0.56      0.243
Speed: 2.5ms preprocess, 46.9ms inference, 0.0ms loss, 18.7ms postprocess per image
Saving D:\prog_py\yolo12_detect\runs\val\val_imgs_512\predictions.json...
Results saved to D:\prog_py\yolo12_d

KeyboardInterrupt: 

##　出力変換

In [1]:
from ultralytics import YOLO

# Load a model

model = YOLO(r"D:\prog_py\Yolo_trainer\runs\train\train9\weights\best.pt")  # load a custom trained model

# Export the model
model.export(format="onnx",
             imgsz=800)

KeyboardInterrupt: 

## auto label

In [2]:
from ultralytics.data.annotator import auto_annotate

auto_annotate(data=r"Dataset\insta360\train",
              det_model=r"D:\prog_py\yolo12_detect\runs\train\train10\weights\best.pt",
              imgsz=800,
              sam_model="sam2.1_l.pt")


image 1/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\024_01_b.jpg: 800x448 87 pods, 84.3ms
image 2/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\024_02_b.jpg: 800x352 97 pods, 76.5ms
image 3/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\024_02_f.jpg: 800x448 88 pods, 48.3ms
image 4/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\024_03_b.jpg: 800x480 92 pods, 73.7ms


KeyboardInterrupt: 